In [3]:
import gurobipy as grb
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta

capacity=pd.read_excel("Marshall_Room_Capacity_Chart_small.xlsx")
course_enrollment=pd.read_excel("Marshall_Course_Enrollment_1516_1617_small.xlsx")
section_info=pd.read_excel("section information.xlsx")

In [2]:
# What we need to extract from data
# prediction about student registeration for every section
# how many section we need to schedule
# the pattern, time length of each section
# classroom capacity

# Variables
# section of course
I=section_info["section"]
# classroom
J=capacity["Room"]
# pattern of session (less than catagories in origional dataset)
P=["Monday","Tuesday","Wednesday","Thursday","Friday"]
P=pd.Series(i for i in P)
# start time of session (from 8:00am to 9:00pm,so the real classtime is between 8:00am to 10:00pm)
T1={}
time = datetime.timedelta(hours=8,minutes=0, seconds=0)
for i in range(27):
    T1[str(time+timedelta(hours=0.5*i))]=i
T=pd.Series(i for i in range(27))

# students' registeration prediction
student={}
for i in I:
    sum1=sum(course_enrollment.loc[course_enrollment.loc[:,"Section"]==i,"Reg Count"])
    len1=len(course_enrollment.loc[course_enrollment.loc[:,"Section"]==i,"Reg Count"])
    student[i]=round(sum1/len1)
#student=pd.Series(i for i in student)

# seat
seat={}
for index,row in capacity.iterrows():
    seat[row["Room"]]=row["Size"]
#seat=pd.Series(i for i in seat)
    
# pattern of each section
PS={}
index=0
for i in I:
    PS[i]=section_info.loc[section_info.loc[:,"section"]==i,"pattern"].get(index)
    index+=1
#PS=pd.Series(i for i in PS)
    
# timelength of each section
TS={}
index=0
for i in I:
    TS[i]=section_info.loc[section_info.loc[:,"section"]==i,"timelength"].get(index)
    index+=1
#TS=pd.Series(i for i in TS)

In [3]:
# Building model
mod=grb.Model()

# Decision Variable
X={}
for i in I:
    for j in J:
        for p in P:
             for t in T:
                    X[i,j,p,t]=mod.addVar(vtype=grb.GRB.BINARY, name='x[{0},{1},{2},{3}]'.format(i,j,p,t))

# Objective
# As i set the constraint1 below, so the section with 2 or 3 sessions will have higher weight here
# but if i do not set constraint1, we cannot promise other sections will not take the vacancy
# for example, if DSO570 assigned to M/W 12:00, namely X[DSO570,j,"MW",12:00]=1, but X[i,j,"M",12:00]=0 and X[i,j,"W",12:00]=0, 
# so other courses maight be assign to this time too
mod.setObjective(sum(X[i,j,p,t]*student[i]/seat[j] for i in I for j in J for p in P for t in T),sense=grb.GRB.MAXIMIZE)


In [4]:
# constraints
I_twoday=pd.Series()
I_oneday=pd.Series()
for i in I:
    if PS.get(i)==1:
        I_oneday=I_oneday.append(pd.Series(i))
    else:
        I_twoday=I_twoday.append(pd.Series(i))

In [5]:
# constraint2
# for X[A course,j,t,p]=1 and if it is 1.5 hours, so for all i expect A course their X[i,j,t,p] should all be 0
# So we have 
# (1) X[A course,j,p,t]+X[i,j,p,t]<=1
# (2) X[A course,j,p,t]+X[i,j,p,t+1]<=1
# (3) X[A course,j,p,t]+X[i,j,p,t+2]<=1
# if it is 3 hours class so we have 6 constraints
# in same time in same classroom in same day there is only one class, no conflicts
# this constraint2 is only for those sections whose pattern is 1, which means they only have one session every week.
constraint2={}
num=0
for i1 in I:
    for j in J:
        for t in T:
            for p in P:
                for i2 in I.drop(pd.Index(I).get_loc(i1)):
                    constraint2[i1,j,p,t,i2,num]=mod.addConstr(X[i1,j,p,t]+X[i2,j,p,t]<=1)

In [6]:
# in the next (number of required sessions)*2 timeslot all of X should be 0
for i1 in I:
    for j in J:
        for t in T:
            for p in P:
                for i2 in I:
                    timeslots=int(TS[i1]*2)
                    if timeslots==6:# 3hour session
                        if t <=21:
                            for num in range(1,timeslots):
                                constraint2[i1,j,p,t,i2,num]=mod.addConstr(X[i1,j,p,t]+X[i2,j,p,t+num]<=1)
                    elif timeslots==4:# 2 hour session
                        if t <=23:
                            for num in range(1,timeslots):
                                constraint2[i1,j,p,t,i2,num]=mod.addConstr(X[i1,j,p,t]+X[i2,j,p,t+num]<=1)
                    elif timeslots==3:# 1.5 hours session
                        if t <=24:
                            for num in range(1,timeslots):
                                constraint2[i1,j,p,t,i2,num]=mod.addConstr(X[i1,j,p,t]+X[i2,j,p,t+num]<=1)
                    elif timeslots==2:# 1 hour session 
                        if t <=25:
                            for num in range(1,timeslots):
                                constraint2[i1,j,p,t,i2,num]=mod.addConstr(X[i1,j,p,t]+X[i2,j,p,t+num]<=1)

In [7]:
# this constraint3 is only for those sections whose pattern is 2, which means they have two session every week.
constraint3={}
for i in I_twoday:
    for j in J:
        for t in T:
            constraint3[i,j,"M",t]=mod.addConstr(X[i,j,"Wednesday",t]== X[i,j,"Monday",t],name="") 
            # the section with two sessions must have same 0/1 in M and W
            
            constraint3[i,j,"T",t]=mod.addConstr(X[i,j,"Thursday",t] == X[i,j,"Tuesday",t],name="") 
            # the section with two sessions must have same 0/1 in M and W

In [8]:
# this constraint4 is to control that when we maximum the utilization rate,the number of student registered 
#for one section should less than the classroom capacity
constraint4={}
for i in I:
    for j in J:
        for p in P:
             for t in T:
                    constraint4[i,j,p,t]=mod.addConstr(X[i,j,p,t]*student[i]<=seat[j],name="")

In [9]:
# this constraint5 is to make sure that in all time in all classroom in all day, 
#for each section, sum(X)=required sessions
constraint5={}
for i in I:
    constraint5[i]=mod.addConstr(sum(X[i,j,p,t] for j in J for p in P for t in T)==PS.get(i),name="")    

In [10]:
#constraint6: No Friday class for MW and TH section
constraint6={}
for i in I_twoday:
    constraint6[i] = mod.addConstr(sum(X[i,j,"Friday",t] for j in J for t in T) == 0) 

In [11]:
#no class can last longer than the '9PM-9.30PM' block
constraint7={}
for i in I:
    constraint7[i] = mod.addConstr(sum(X[i,j,p,T.max()-ts] for j in J for p in P for ts in range(int(TS[i]*2))) == 0)

In [12]:
mod.optimize()
mod.setParam('OutputFlag',False)

print('Optimal solution:',mod.ObjVal)

Optimize a model with 16597 rows, 1080 columns and 33400 nonzeros
Variable types: 0 continuous, 1080 integer (1080 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [7e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective 4.0416667
Presolve removed 16483 rows and 945 columns
Presolve time: 0.04s
Presolved: 114 rows, 135 columns, 677 nonzeros
Found heuristic solution: objective 4.3685897
Variable types: 0 continuous, 135 integer (135 binary)

Root relaxation: cutoff, 2 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0         4.36859    4.36859  0.00%     -    0s

Explored 0 nodes (2 simplex iterations) in 0.10 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 4.36859 

Optimal solution found (tolerance 1.00e-04)
B

In [13]:
for i in I:
    for j in J:
        for p in P:
            for t in T:
                if X[i,j,p,t].x>0:
                    print("i ",i," j ",j," p ",p," t ",t," X[i,j,p,t] ",X[i,j,p,t].x," student[i] ",student[i]," seat[j] ",seat[j]," X[i,j,p,t]*student[i]/seat[j] ",X[i,j,p,t].x*student[i]/seat[j])

i  14025  j  ACC236  p  Monday  t  0  X[i,j,p,t]  1.0  student[i]  34.0  seat[j]  39  X[i,j,p,t]*student[i]/seat[j]  0.871794871795
i  14025  j  ACC236  p  Wednesday  t  0  X[i,j,p,t]  1.0  student[i]  34.0  seat[j]  39  X[i,j,p,t]*student[i]/seat[j]  0.871794871795
i  14026  j  ACC201  p  Friday  t  20  X[i,j,p,t]  1.0  student[i]  44.0  seat[j]  48  X[i,j,p,t]*student[i]/seat[j]  0.916666666667
i  14052  j  ACC201  p  Tuesday  t  4  X[i,j,p,t]  1.0  student[i]  40.0  seat[j]  48  X[i,j,p,t]*student[i]/seat[j]  0.833333333333
i  14051  j  ACC201  p  Tuesday  t  0  X[i,j,p,t]  1.0  student[i]  42.0  seat[j]  48  X[i,j,p,t]*student[i]/seat[j]  0.875


In [14]:
T1=[]
blank=[]
time = datetime.timedelta(hours=8,minutes=0, seconds=0)
for i in range(27):
    T1.append(str(time+timedelta(hours=0.5*i)))
    blank.append("no class")

In [15]:
schedule=pd.DataFrame({"# timeslot": range(27),
                       "timeslot":T1,
                       "Monday":blank,
                       "Tuesday":blank,
                       "Wednesday":blank,
                       "Thursday":blank,
                       "Friday":blank})

sched_cols = ['# timeslot','timeslot','Monday','Tuesday','Wednesday','Thursday','Friday']
schedule = schedule[sched_cols]
schedule=schedule.set_index('# timeslot')

In [16]:
# put the optimal solution into our beautiful schedule table
for i in I:
    for j in J:
        for p in P:
            for t in T:
                if X[i,j,p,t].x>0:
                    section_content="section id: "+str(i)+" classroom: "+str(j)
                    print(section_content)
                    for a in range(int(TS[i]*2)):
                        schedule.loc[t+a,p]=section_content

section id: 14025 classroom: ACC236
section id: 14025 classroom: ACC236
section id: 14026 classroom: ACC201
section id: 14052 classroom: ACC201
section id: 14051 classroom: ACC201


In [17]:
schedule

,timeslot,Monday,Tuesday,Wednesday,Thursday,Friday
# timeslot,,,,,,
0,8:00:00,section id: 14025 classroom: ACC236,section id: 14051 classroom: ACC201,section id: 14025 classroom: ACC236,no class,no class
1,8:30:00,section id: 14025 classroom: ACC236,section id: 14051 classroom: ACC201,section id: 14025 classroom: ACC236,no class,no class
2,9:00:00,section id: 14025 classroom: ACC236,section id: 14051 classroom: ACC201,section id: 14025 classroom: ACC236,no class,no class
3,9:30:00,no class,section id: 14051 classroom: ACC201,no class,no class,no class
4,10:00:00,no class,section id: 14052 classroom: ACC201,no class,no class,no class
5,10:30:00,no class,section id: 14052 classroom: ACC201,no class,no class,no class
6,11:00:00,no class,section id: 14052 classroom: ACC201,no class,no class,no class
7,11:30:00,no class,section id: 14052 classroom: ACC201,no class,no class,no class
8,12:00:00,no class,no class,no class,no class,no class
